In [1]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ScierKnave/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-x_shiye0
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-x_shiye0'
c:\ProgramData\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Hyperparameters

In [2]:
# MPS

# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 20
# USING CUSTOM FEATURE MAP WITH FORK?: YES

# Training parameters
nepochs_student_HP = 25 
student_lr_HP = 1e-4
student_reg_HP = 0
student_loss_HP = nn.CrossEntropyLoss()


# Premilinaries: Importing the data and utils subroutines

In [4]:
# Import the mnist train dataset
train_set = torch.load('datasets/train_mnist.pt')

train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    batch_size=batch_sz_HP
    )

# Import the mnist test set
test_set = torch.load('datasets/test_mnist.pt')
# Create a testing batch iterator
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    batch_size = batch_sz_HP
    )

In [ ]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / nb_test_HP # divide by total size

# Training the student model

In [ ]:

# Initialize the MPS module
student = MPS(
    input_dim = 28 ** 2,
    output_dim = 10,
    bond_dim = bond_dim_HP
).to(chosen_device) 
#student.register_feature_map(feature_map_HP)

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(
    student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)

# Used on the inputs before the loss function
LogSoftmax = nn.LogSoftmax(dim=1)

# Create an array to store the val loss
# of the student at each epoch
stud_test_loss = []
stud_train_loss = []

softmax = nn.Softmax(dim=1)
# Training loop 
for epoch in range(nepochs_student_HP):
    for (x_mb, y_mb) in train_iterator:
        # Flatten the MNIST images, which come in matrix form
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        student_logits = student(x_mb) 

        # Backpropagation
        loss = student_loss_HP( student_logits, y_mb)

        student_optimizer.zero_grad()
        loss.backward()
        student_optimizer.step()

    print(loss)

    #stud_train_loss.append( round(get_acc(student, train_iterator).item(), 3) )
    stud_test_loss.append( round( get_acc(student, test_iterator).item(), 5) )
    print(stud_test_loss)
    print(stud_train_loss)

print("Epochs: ", np.arange(1, nepochs_student_HP+1).tolist())
print("Train loss: ", stud_train_loss)
print("Test loss: ", stud_test_loss)


tensor(2.2544, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1]
[]
tensor(1.8736, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743]
[]
tensor(1.0404, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932]
[]
tensor(0.6320, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932, 0.58134]
[]
tensor(0.7699, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932, 0.58134, 0.68256]
[]
tensor(0.8187, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932, 0.58134, 0.68256, 0.78223]
[]
tensor(0.5306, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932, 0.58134, 0.68256, 0.78223, 0.81431]
[]
tensor(0.4838, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932, 0.58134, 0.68256, 0.78223, 0.81431, 0.82391]
[]
tensor(0.3452, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.31743, 0.53932, 0.58134, 0.68256, 0.78223, 0.81431, 0.82391, 0.82482]
[]
tensor(0.4764, device='cuda:0', grad_fn=<NllLossBackwar